In [2]:
import tensorflow as tf
import os
sess=tf.Session()    
#First let's load meta graph and restore weights
os.chdir("/Users/luoxin/Desktop/[AML]Advanced Machine Learning/final_proj/atari")
from run_dqn_atari import *
saver = tf.train.import_meta_graph('pong.meta')

In [4]:
ckpt = tf.train.get_checkpoint_state('')
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)

INFO:tensorflow:Restoring parameters from pong


In [8]:
q_func_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='q_func')
target_q_func_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_q_func')

In [ ]:
# Get Atari games.
benchmark = gym.benchmark_spec('Atari40M')
# Change the index to select a different game.
task = benchmark.tasks[3]
# Run training
seed = 0 # Use a seed of zero (you may want to randomize the seed!)
env = get_env(task, seed)
session = get_session()
